In [1]:
import utils
from importlib import reload
import os
import torch
import numpy as np
import tqdm
from gemlite.core import GemLiteLinearTriton
reload(utils)

/home/anon/miniconda3/envs/neurips_2025/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loaded /home/anon/miniconda3/envs/neurips_2025/lib/python3.12/site-packages/gemlite/configs/4090.json config.


<module 'utils' from '/home/anon/phd_master/neurips_2025/clean/utils.py'>

In [2]:
base_checkpoint_path = "../checkpoints"

# Set to false if you want to benchamrk the base model without double binary factorization
dbf = True
experiment_id = 0
LEN_TOKENS_FOR_GENERATION = 128
REPETITIONS = 100

# We provide triton autotune config. It saves the time during autotuning,
# but the results may not be optimal for your GPU.
load_triton_autotune_config = True

experiments = [
    ("latest_llama2-1b-10.pt", "meta-llama/Llama-2-7b-hf"),
    ("latest_llama2-1b-15.pt", "meta-llama/Llama-2-7b-hf"),
    ("latest_llama2-1b-20.pt", "meta-llama/Llama-2-7b-hf"),
    ("latest_llama2-1b-23.pt", "meta-llama/Llama-2-7b-hf"),
    ("latest_llama31b10.pt", "meta-llama/Meta-Llama-3-8B"),
    ("latest_llama31b15.pt", "meta-llama/Meta-Llama-3-8B"),
    ("latest_llama31b20.pt", "meta-llama/Meta-Llama-3-8B"),
    ("latest_llama31b23.pt", "meta-llama/Meta-Llama-3-8B"),
]
dbp_checkpoint_name, base_model_str = experiments[experiment_id]

checkpoint_path = os.path.join(base_checkpoint_path, dbp_checkpoint_name)

In [3]:
if load_triton_autotune_config:
    GemLiteLinearTriton.load_config(f'test_config_full_eval.json')

In [4]:
tokenizer = utils.get_tokezizer(base_model_str)
input_ids = tokenizer("", return_tensors="pt").to("cuda")
dbf_weights = torch.load(checkpoint_path)


In [5]:
%%time
model = utils.get_llama(base_model_str)

Loading checkpoint shards: 100%|███████████████████████████████████████| 2/2 [00:02<00:00,  1.37s/it]

CPU times: user 1.66 s, sys: 1.28 s, total: 2.94 s
Wall time: 3.39 s


In [6]:
reload(utils)

<module 'utils' from '/home/anon/phd_master/neurips_2025/clean/utils.py'>

In [7]:
%%time
if dbf:
    for i in range(32):
        print(i)
        att = model.model.layers[i].self_attn
        mlp = model.model.layers[i].mlp
        
        att.q_proj = utils.DoubleBinaryLinear(dbf_weights, i, "self_attn.q_proj")
        att.k_proj = utils.DoubleBinaryLinear(dbf_weights, i, "self_attn.k_proj")
        att.v_proj = utils.DoubleBinaryLinear(dbf_weights, i, "self_attn.v_proj")
        att.o_proj = utils.DoubleBinaryLinear(dbf_weights, i, "self_attn.o_proj")
        mlp.gate_proj = utils.DoubleBinaryLinear(dbf_weights, i, "mlp.gate_proj")
        mlp.up_proj = utils.DoubleBinaryLinear(dbf_weights, i, "mlp.up_proj")
        mlp.down_proj = utils.DoubleBinaryLinear(dbf_weights, i, "mlp.down_proj")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
CPU times: user 2min 58s, sys: 49.3 s, total: 3min 47s
Wall time: 3min 4s


In [8]:
generator = utils.CustomGenerator(model, tokenizer, LEN_TOKENS_FOR_GENERATION)


In [9]:
%%time
# First generate is slow because of compilation
out = generator.generate()

/home/anon/miniconda3/envs/neurips_2025/lib/python3.12/site-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


CPU times: user 54.6 s, sys: 16.6 s, total: 1min 11s
Wall time: 1min 12s


In [10]:
%%time
out = generator.generate()

CPU times: user 643 ms, sys: 7.9 ms, total: 651 ms
Wall time: 650 ms


In [11]:
print(utils.to_text(out, tokenizer))

Return to Forest From Panels From Cages Trials Divilation 19.ng/ Library
Home › Collections › Forestmah › Forest ›
NEWSPAN_29-12-2012_PH4288.jpg (145.4 KB, 173 W) Add: 1 (1.06 MB) Click here to find the college's WGC page. Download
Going full
(94.25 MB, 15.99 MB) Added: 10 (9


In [12]:
t_milis = []
for _ in tqdm.tqdm(range(REPETITIONS)):
    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)
    start_event.record()
    out = generator.generate()
    end_event.record()
    torch.cuda.synchronize()
    t_milis.append(start_event.elapsed_time(end_event))

np.min(t_milis), np.mean(t_milis[-5:]), np.max(t_milis)

100%|██████████████████████████████████████████████████████████████| 100/100 [01:05<00:00,  1.54it/s]


(np.float64(647.1263427734375),
 np.float64(650.03896484375),
 np.float64(656.0706176757812))

In [13]:
inference_time = np.mean(t_milis[-5:])
print(f"Inference time {inference_time} milis")
print(f"Tokens/s = {LEN_TOKENS_FOR_GENERATION/(inference_time/1000)}")

Inference time 650.03896484375 milis
Tokens/s = 196.91127289695223
